# Planner calculations

In [57]:
# Import necessary libraries
import xml.etree.ElementTree as ET
import pandas as pd

# Function to parse tasks with WBS
def parse_tasks_no_duplicates(tasks, parent_wbs="", top_level_counter=[1], processed_ids=set()):
    wbs_list = []
    local_counter = 1

    for task in tasks:
        task_id = task.get("id", "")
        if task_id in processed_ids:  # Skip duplicates
            continue
        processed_ids.add(task_id)

        name = task.get("name", "")
        duration = task.get("duration", "0")
        start = task.get("start", "")[:8]  # Trimming to YYYYMMDD format
        end = task.get("end", "")[:8]     # Trimming to YYYYMMDD format

        # Generate WBS
        if parent_wbs:
            current_wbs = f"{parent_wbs}.{local_counter}"
            local_counter += 1
        else:  # Top-level task
            current_wbs = str(top_level_counter[0])
            top_level_counter[0] += 1

        # Append task data
        wbs_list.append({
            "WBS": current_wbs,
            "ID": task_id,
            "Name": name,
            "Duration (Workdays)": round(int(duration) / (8 * 3600),0),
            "Start": start,
            "End": end,
            "Milestone": int(duration) == 0
        })

        # Recurse into child tasks
        child_tasks = task.findall("./task")
        if child_tasks:
            wbs_list.extend(parse_tasks_no_duplicates(
                child_tasks, parent_wbs=current_wbs, top_level_counter=top_level_counter, processed_ids=processed_ids
            ))

    return wbs_list

# Load XML file
xml_file_path = "CZPAD.planner"  # Replace with the path to your file
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Parse tasks and create a DataFrame
root_tasks = root.findall(".//task")
parsed_tasks = parse_tasks_no_duplicates(root_tasks)
tasks_df = pd.DataFrame(parsed_tasks)

# Convert Start and End to datetime
tasks_df["Start"] = pd.to_datetime(tasks_df["Start"], format="%Y%m%d")
tasks_df["End"] = pd.to_datetime(tasks_df["End"], format="%Y%m%d")

# Display the resulting DataFrame (optional)
tasks_df


,WBS,ID,Name,Duration (Workdays),Start,End,Milestone
0,1,1,KO,0.0,2025-04-01,2025-04-01,True
1,2,2,Analysis and Design,80.0,2025-04-01,2025-07-21,False
2,2.1,3,Review of current FM (TRL9),20.0,2025-04-01,2025-04-28,False
3,2.2,4,Mechanical and thermal redesign,30.0,2025-04-28,2025-06-09,False
4,2.3,5,Electrical redesign,30.0,2025-04-28,2025-06-09,False
5,2.4,6,Wearing design,30.0,2025-06-09,2025-07-21,False
6,2.5,7,SRR (System Requirement Review),0.0,2025-06-09,2025-06-09,True
7,3,8,Manufacturing and Testing EM,90.0,2025-06-09,2025-10-13,False
8,3.1,9,EM manufacturing (TRL7),30.0,2025-06-09,2025-07-21,False
9,3.2,10,Battery module EQM manufacturing (TRL6),30.0,2025-06-09,2025-07-21,False


In [58]:
tasks_df.to_csv('tasks.csv')

In [48]:
import xml.etree.ElementTree as ET
import pandas as pd

# Load the XML file
xml_file_path = "CZPAD.planner"
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Parse tasks from XML
def parse_tasks_with_wbs(tasks, parent_wbs="", top_level_counter=[1], processed_ids=set()):
    wbs_list = []
    local_counter = 1

    for task in tasks:
        task_id = task.get("id", "")
        if task_id in processed_ids:  # Skip duplicates
            continue
        processed_ids.add(task_id)

        name = task.get("name", "")
        duration = task.get("duration", "0")
        start = task.get("start", "")[:8]  # Trimming to YYYYMMDD format
        end = task.get("end", "")[:8]     # Trimming to YYYYMMDD format

        # Generate WBS
        if parent_wbs:
            current_wbs = f"{parent_wbs}.{local_counter}"
            local_counter += 1
        else:  # Top-level task
            current_wbs = str(top_level_counter[0])
            top_level_counter[0] += 1

        # Append task data
        wbs_list.append({
            "task-id": task_id,
            "name": name,
            "duration": duration,
            "start": start,
            "end": end,
            "WBS": current_wbs
        })

        # Recurse into child tasks
        child_tasks = task.findall("./task")
        if child_tasks:
            wbs_list.extend(parse_tasks_with_wbs(
                child_tasks, parent_wbs=current_wbs, top_level_counter=top_level_counter, processed_ids=processed_ids
            ))

    return wbs_list

# Parse tasks
root_tasks = root.findall(".//task")
tasks_data = parse_tasks_with_wbs(root_tasks)
tasks_with_wbs_df = pd.DataFrame(tasks_data)

# Parse allocations and calculate workdays
def parse_allocations_with_durations(root, tasks_df):
    task_duration_map = tasks_df.set_index("task-id")["duration"].astype(float) / (8 * 3600)  # Convert seconds to workdays
    
    allocations = []
    for allocation in root.findall(".//allocation"):
        task_id = allocation.get("task-id")
        resource_id = allocation.get("resource-id")
        units = float(allocation.get("units", "0")) / 100.0  # Convert percentage to fraction
        
        if task_id in task_duration_map.index:
            allocations.append({
                "task-id": task_id,
                "resource-id": resource_id,
                "units": units,
                "workdays": round(units * task_duration_map[task_id],0)  # Calculate resource workdays
            })

    return pd.DataFrame(allocations)

# Parse allocations with durations
allocations_with_workdays_df = parse_allocations_with_durations(root, tasks_with_wbs_df)

# Pivot the table to have resource-ids as columns
allocations_pivot = allocations_with_workdays_df.pivot_table(
    index="task-id", 
    columns="resource-id", 
    values="workdays", 
    fill_value=0
)

# Rename columns based on short-name
resources_df = pd.DataFrame([
    {"resource-id": res.get("id"), "short-name": res.get("short-name")}
    for res in root.findall(".//resource")
])
resource_map = resources_df.set_index("resource-id")["short-name"].to_dict()
allocations_pivot.columns = [resource_map.get(col, f"Resource-{col}") for col in allocations_pivot.columns]

# Add WBS to the table
allocations_pivot["WBS"] = tasks_with_wbs_df.set_index("task-id")["WBS"]

# Reorder columns to place WBS at the beginning
columns = ["WBS"] + [col for col in allocations_pivot.columns if col != "WBS"]
allocations_df = allocations_pivot[columns]

# Ensure task-id is numeric and sort
allocations_df = allocations_df.reset_index()
allocations_df["task-id"] = allocations_df["task-id"].astype(int)
allocations_df = allocations_df.sort_values(by="task-id").set_index("task-id")

# The resulting dataframe is named `allocations_df`
allocations_df


,WBS,EE,PROJ,UST,PR,ME,SCI,FW,SUP
task-id,,,,,,,,,
3,2.1,10,0,0,0,10,10,0,0
4,2.2,0,0,0,0,30,0,0,0
5,2.3,30,0,0,0,0,0,0,0
6,2.4,0,0,0,0,30,0,0,0
9,3.1,0,0,30,0,0,0,0,0
10,3.2,0,0,30,0,0,0,0,0
11,3.3,0,0,0,0,0,0,60,0
12,3.4,30,0,0,0,30,0,0,0
15,4.1,0,0,30,0,0,0,0,0


In [49]:
# Extract the first digit of the WBS and create a grouping column
allocations_df["WBS_Group"] = allocations_df["WBS"].str.split('.').str[0]

# Group by the first digit of the WBS and sum rows for each group
grouped_allocations = allocations_df.groupby("WBS_Group").sum(numeric_only=True)

# Drop the grouping column if not needed in the final output
grouped_allocations = grouped_allocations.drop(columns=["WBS_Group"], errors="ignore")

# Display the grouped DataFrame
print(grouped_allocations)


           EE  PROJ  UST   PR  ME  SCI  FW  SUP
WBS_Group                                      
2          40     0    0    0  70   10   0    0
3          30     0   60    0  30    0  60    0
4          46     0   90    0  60   30   0   60
5          40     0   30    0  40   10  60   30
6          10     0    0    0  10  170   0   44
7           0     0    0  281   0   40   0    0
8           0   280    0    0   0    0   0    0


In [55]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = "team_structure.csv"
team = pd.read_csv(file_path)

# Display the DataFrame for verification
print(team)


    team    team-leader           team-member  FTE Company
0     EE   Jakub Kakona          Jakub Kakona  1.0     UST
1     EE   Jakub Kakona  Electrical Engineer2  1.0     UST
2     ME   Roman Dvorak          Roman Dvorak  1.0     UST
3     ME   Roman Dvorak  Mechanical Engineer2  0.5     UST
4    SCI    Ondrej Ploc           Ondrej Ploc  0.5     NPI
5    SCI    Ondrej Ploc         Iva Ambrozova  0.5     NPI
6    SCI    Ondrej Ploc         Martin Kakona  0.8     NPI
7    SCI    Ondrej Ploc           Jakub Slegl  0.5     NPI
8     FW   Vit Hanousek          Vit Hanousek  1.0     UST
9     FW   Vit Hanousek          Programator2  0.5     UST
10   SUP  Iva Ambrozova           Technician1  0.5     NPI
11   SUP  Iva Ambrozova           Technician2  0.5     NPI
12  PROJ  Iva Ambrozova         Iva Ambrozova  0.3     NPI
13  PROJ  Iva Ambrozova                Roxana  0.5     NPI
14   UST   Jakub Kakona                   NaN  NaN     NaN
15    PR  Iva Ambrozova            PR manager  0.2     N

In [51]:
# Initialize a list to store results
workdays_per_member = []

# Iterate over each WBS_Group
for wbs_group, row in grouped_allocations.iterrows():
    # Iterate over each team
    for team_name, total_workdays in row.items():
        if total_workdays > 0:  # Check if there are workdays assigned
            # Get the members of the current team
            team_members = team[team["team"] == team_name]
            for _, member in team_members.iterrows():
                # Calculate workdays for each member
                member_workdays = round(total_workdays * member["FTE"],0)
                workdays_per_member.append({
                    "WBS_Group": wbs_group,
                    "Team": team_name,
                    "Team Member": member["team-member"],
                    "Workdays": member_workdays
                })

# Convert to DataFrame
workdays_per_member_df = pd.DataFrame(workdays_per_member)

# Display specific results for Jakub Kakona in WBS_Group 2
jakub_kakona_results = workdays_per_member_df[
    (workdays_per_member_df["Team Member"] == "Jakub Kakona") & 
    (workdays_per_member_df["WBS_Group"] == "2")
]

workdays_per_member_df

,WBS_Group,Team,Team Member,Workdays
0,2,EE,Jakub Kakona,40.0
1,2,EE,Electrical Engineer2,40.0
2,2,ME,Roman Dvorak,70.0
3,2,ME,Mechanical Engineer2,35.0
4,2,SCI,Ondrej Ploc,5.0
5,2,SCI,Iva Ambrozova,5.0
6,2,SCI,Martin Kakona,8.0
7,2,SCI,Jakub Slegl,5.0
8,3,EE,Jakub Kakona,30.0
9,3,EE,Electrical Engineer2,30.0


In [52]:
workdays_per_member_df.to_csv('workdays.csv')

In [53]:
# Group by team member and sum their workdays
total_workdays_per_member = workdays_per_member_df.groupby("Team Member")["Workdays"].sum().reset_index()

# Rename columns for clarity
total_workdays_per_member.columns = ["Team Member", "Total Workdays"]

# Display the resulting DataFrame
total_workdays_per_member

,Team Member,Total Workdays
0,Electrical Engineer2,166.0
1,Iva Ambrozova,214.0
2,Jakub Kakona,166.0
3,Jakub Slegl,130.0
4,Martin Kakona,208.0
5,Mechanical Engineer2,105.0
6,Ondrej Ploc,130.0
7,PR manager,56.0
8,Programator2,60.0
9,Roman Dvorak,210.0


In [54]:
total_workdays_per_member.to_csv('total_workdays.csv')